In [2]:
# using Pkg
# Pkg.add(url="https://github.com/MasanoriKanamaru/Astroshaper")
# Pkg.update("Astroshaper")
# Pkg.test("Astroshaper")

In [3]:
using Plots
using DataFrames
using LinearAlgebra
using Statistics
using Parameters
# using FileIO
# using JLD2
# using Profile

In [4]:
using Revise
using Astroshaper

┌ Info: Precompiling Astroshaper [ab46f4a3-3c83-4d86-98d6-41a8b1a2e76e]
└ @ Base loading.jl:1423


In [5]:
ENV["COLUMNS"] = 1000
ENV["LINES"] = 20;

In [6]:
shapedir = "/Users/masanorikanamaru/Documents/shape/ryugu"

shapename = "ryugu_v252_f500.obj"
# shapename = "ryugu_v752_f1500.obj"
# shapename = "ryugu_test.obj"

# shapename = "SHAPE_SPC_49k_v20190802.obj"
# shapename = "SHAPE_SPC_200k_v20190802.obj"
# shapename = "SHAPE_SPC_800k_v20190802.obj"
# shapename = "SHAPE_SPC_3M_v20190802.obj"

# shapename = "SHAPE_SFM_49k_v20180804.obj"
# shapename = "SHAPE_SPC_49k_v20200323.obj"
# shapename = "SHAPE_SPC_3M_v20200323.obj"

shapepath = joinpath(shapedir, shapename)
@show shapepath;

shapepath = "/Users/masanorikanamaru/Documents/shape/ryugu/ryugu_v252_f500.obj"


In [7]:
# shapepath = splitext(shapepath)[1] * ".jld2"

In [9]:
shape = ShapeModel(shapepath; scale=1000, find_visible_facets=true, save_shape=false)

Shape model
-----------
Nodes             : 252
Faces             : 500
Surface area      : 2.6740222482370236e6
Volume            : 3.759699719284749e8
Equivalent radius : 447.7354864412234
Center-of-Figure  : [0.37115829680556717, 0.14301318037402377, 0.02972401268611568]
Inertia tensor    : 
    | Ixx Ixy Ixz |   [0.0, 0.0, 0.0]
    | Iyx Iyy Iyz | = [0.0, 0.0, 0.0]
    | Izx Izy Izz |   [0.0, 0.0, 0.0]


In [25]:
# draw(shape)
# draw(shape; strokecolor=:gray10)
# draw(shape, data=:radius)
draw(shape, data=:temperature, colormap=:heat)
# draw(shape; data=:illumination, r̂☉=[1,0,0.])
# draw(shape, data=[center[3] for center in shape.facets.center])

GLMakie.Screen(...)

In [26]:
orbit = OrbitalElements(RYUGU)

Orbital elements
----------------


Semi-mojor axis        : a  = 1.18956373 [AU]
Eccentricity           : e  = 0.19027921 [-]
Lon. of ascending node : Ω  = 251.589203 [deg]
Argument of periapsis  : ω  = 211.435963 [deg]
Inclination            : I  = 5.8840222 [deg]
Periapsis passage time : tₚ = -2.4948038004043424e6 [sec]
Mean anomaly           : Φ  = 21.9353799 [deg]

Other parameters
----------------
Gravitational parameter : μ = 1.32712440018e20 [m^3/s^2]
Mean motion             : n = 0.7596656791419168 [deg/day]
Orbital period          : T = 473.8926739544681 [day]


In [27]:
spin = SpinParams(RYUGU, orbit)

Spin parameters
---------------


Right ascension (RA) : α = 96.4 [deg]
Declination (Dec)    : δ = -66.4 [deg]
Ecliptic longitude   : λ = 179.33757188938182 [deg]
Ecliptic latitude    : β = -87.44207056697601 [deg]
Obliquity            : ε = 171.65664649001607 [deg]
Spin period          : P = 7.63262 [hours]
Spin rate            : ω = 0.00022866712242903872 [rad/sec]
Vernal equinox lon.  : γ = 154.3323208882217 [deg]
                           (longitude from the periheion direction)


# Yarkovsky効果

## Yarkovsky drift rate vs. Bond albedo

In [15]:
t_end = orbit.T + spin.P * 150

thermo_params = ThermoParams(
    # A_B   = 0.00,
    A_B   = 0.04,
    # A_B   = 0.10,
    # A_B   = 0.20,
    # A_B   = 0.30,
    # A_B   = 0.40,
    # A_B   = 0.50,
    # A_B   = 0.60,
    # A_B   = 0.70,
    # A_B   = 0.80,
    # A_B   = 0.90,
    # A_B   = 0.10,

    A_TH  = 0.0,
    k     = 0.1,
    ρ     = 1270.0,
    Cp    = 600.0,
    ϵ     = 1.,
    t_bgn = 0.0,
    t_end = t_end,
    Nt    = length(0 : spin.P/72 : t_end),
    z_max = 0.6,
    Nz    = 41,
    P     = spin.P,
)

Thermophysical parameters
-------------------------
A_B   : 0.04
A_TH  : 0.0
k     : 0.1
ρ     : 1270.0
Cp    : 600.0
ϵ     : 1.0
-------------------------
t_bgn : 0.0
t_bgn : 0.0 (Normalized by period P)
t_end : 4.506594182966604e7
t_end : 1640.1074827395094 (Normalized by period P)
Nt    : 118088
Δt    : 381.6333874996066
Δt    : 0.013888975778362643 (Normalized by period P)
-------------------------
z_max : 0.6
z_max : 2.8186148335177874 (Normalized by skin depth l)
Nz    : 41
Δz    : 0.015
Δz    : 0.07046537083794469 (Normalized by skin depth l)
-------------------------
P     : 27477.432
l     : 0.21287051812296282
Γ     : 276.04347483684523
λ     : 0.2225916520849266
-------------------------


In [64]:
@time df = run_TPM(shape, orbit, spin, thermo_params)

# df[:, :ȧ] = [ Astroshaper.drift_rate_ȧ([row.f_x, row.f_y, row.f_z] / RYUGU[:M], row.u, orbit) for row in eachrow(df) ]
# ȧ_mean = mean( df.ȧ[@. df.t > (df.t[end] - orbit.T)] ) * 3600 * 24 * 365

  0.235123 seconds (41.10 k allocations: 7.649 MiB)


,t,u,ϕ,f_x,f_y,f_z,τ_x,τ_y,τ_z,E_in,E_out,ν,E_cons,Ē_cons
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.468819,0.0,9.92025e-6,4.44963e-6,6.01218e-7,-0.000416175,0.000149287,-0.000183821,8.38911e8,0.0,0.563643,0.0,0.0
2,381.631,0.46889,0.0872665,0.0251434,0.0132682,-0.00181507,0.37064,-0.618929,-0.19855,8.3566e8,1.68572e7,0.563727,0.0201723,0.0100862
3,763.262,0.46896,0.174533,0.0510587,0.025989,-0.00369733,0.70619,-1.15349,-0.382106,8.34888e8,3.4082e7,0.56381,0.0408222,0.0203315
4,1144.89,0.469031,0.261799,0.0787493,0.0381852,-0.00567068,1.03461,-1.65732,-0.55864,8.35013e8,5.22418e7,0.563894,0.062564,0.0308896
5,1526.52,0.469101,0.349066,0.106405,0.0487858,-0.00763911,1.35364,-2.10314,-0.708673,8.34828e8,7.00753e7,0.563977,0.0839398,0.0414997
6,1908.15,0.469172,0.436332,0.133083,0.057346,-0.00956968,1.67198,-2.49911,-0.831672,8.33457e8,8.69602e7,0.56406,0.104337,0.0519725
7,2289.79,0.469242,0.523599,0.158207,0.0637697,-0.0114638,1.99403,-2.86807,-0.930422,8.38661e8,1.02548e8,0.564144,0.122276,0.0620158
8,2671.42,0.469313,0.610865,0.181472,0.0679746,-0.0133198,2.29748,-3.2293,-1.03886,8.41196e8,1.16765e8,0.564227,0.138809,0.071615
9,3053.05,0.469383,0.698132,0.202615,0.0702689,-0.0151363,2.57798,-3.60047,-1.1569,8.43099e8,1.29508e8,0.564311,0.153609,0.0807254


In [65]:
describe(df)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,t,1.37387e5,0.0,1.37387e5,2.74774e5,0,Float64
2,u,0.494167,0.468819,0.494179,0.519469,0,Float64
3,ϕ,31.4159,0.0,31.4159,62.8319,0,Float64
4,f_x,0.555033,9.92025e-6,0.588482,0.642349,0,Float64
5,f_y,0.152681,4.44963e-6,0.161908,0.218619,0,Float64
6,f_z,-0.0679187,-0.102933,-0.0737024,6.01218e-7,0,Float64
7,τ_x,2.37387,-1.18597,2.24983,6.10219,0,Float64
8,τ_y,-8.24232,-11.2672,-8.11157,0.000149287,0,Float64
9,τ_z,-0.737358,-5.6173,-1.3916,7.35745,0,Float64


In [26]:
#### Bond albedo を変えた場合 ####
# 全球で A_B = 0.00 : -144.45409088016052
# 全球で A_B = 0.04 : -138.61342928209942  # Ryugu default
# 全球で A_B = 0.10 : -129.7704183627798
# 全球で A_B = 0.20 : -114.81121874363524
# 全球で A_B = 0.30 : -99.57816555321362
# 全球で A_B = 0.40 : -84.08719629669288
# 全球で A_B = 0.50 : -68.38164025827987
# 全球で A_B = 0.60 : -52.55894649198356
# 全球で A_B = 0.70 : -36.82941437263273
# 全球で A_B = 0.80 : -21.660545349731272
# 全球で A_B = 0.90 : -8.203443369183109
# 全球で A_B = 0.95 : -3.0141934949134384
# 全球で A_B = 1.00 : -0.28995137028523116

In [20]:
# df = DataFrame(A_B=Float64[], ȧ_mean=Float64[])

# push!(df, (0.00, -144.45409088016052))
# push!(df, (0.04, -138.61342928209942))
# push!(df, (0.10, -129.7704183627798))
# push!(df, (0.20, -114.81121874363524))
# push!(df, (0.30, -99.57816555321362))
# push!(df, (0.40, -84.08719629669288))
# push!(df, (0.50, -68.38164025827987))
# push!(df, (0.60, -52.55894649198356))
# push!(df, (0.70, -36.82941437263273))
# push!(df, (0.80, -21.660545349731272))
# push!(df, (0.90, -8.203443369183109))
# push!(df, (0.95, -3.0141934949134384))
# push!(df, (1.00, -0.28995137028523116))

# plot(framestyle=:box)
# plot!(legend=false)

# plot!(df.A_B, df.ȧ_mean, lw=5, lc=:orange)
# scatter!(df.A_B, df.ȧ_mean, ms=5, msw=1, mc=:orange)

# plot!(xlims=(0, 1))
# plot!(ylims=(-150, 0))
# plot!(xlabel="Bond albedo")
# plot!(ylabel="Yarkovsky drift rate, da/dt [m/yr]")

# plot!(size=(400,300))
# # savefig("Yarkovsky_vs_albedo.pdf")

## 半球を白く塗る
- デフォルト： `A_B = 0.04`
- 半球　　　： `A_B = 0.90`

In [102]:
t_end = orbit.T + spin.P * 150

thermo_params = ThermoParams(
    A_B   = fill(0.04, shape.num_face),
    A_TH  = 0.0,
    k     = 0.1,
    ρ     = 1270.0,
    Cp    = 600.0,
    ϵ     = 1.,
    t_bgn = 0.0,
    t_end = t_end,
    Nt    = length(0 : spin.P/72 : t_end),
    z_max = 0.6,
    Nz    = 41,
    P     = spin.P,
);

In [103]:
# selected = [center[1] > 0 for center in shape.facets.center]  # x > 0
# selected = [center[1] < 0 for center in shape.facets.center]  # x < 0
# selected = [center[2] > 0 for center in shape.facets.center]  # y > 0
# selected = [center[2] < 0 for center in shape.facets.center]  # y < 0
# selected = [center[3] > 0 for center in shape.facets.center]  # z > 0
selected = [center[3] < 0 for center in shape.facets.center]  # z < 0

thermo_params.A_B[selected] .= 0.9

thermo_params.A_B

1500-element Vector{Float64}:
 0.9
 0.9
 0.9
 0.9
 0.9
 0.9
 0.9
 0.9
 ⋮
 0.04
 0.9
 0.04
 0.9
 0.04
 0.04
 0.04

In [104]:
draw(shape; data=thermo_params.A_B, colormap=:grays)

GLMakie.Screen(...)

In [105]:
@time df = run_TPM(shape, orbit, spin, thermo_params)

df[:, :ȧ] = [ Astroshaper.drift_rate_ȧ([row.f_x, row.f_y, row.f_z] / RYUGU[:M], row.u, orbit) for row in eachrow(df) ]
ȧ_mean = mean( df.ȧ[@. df.t > (df.t[end] - orbit.T)] ) * 3600 * 24 * 365

167.132034 seconds (8.38 M allocations: 4.516 GiB, 0.41% gc time)


-80.89037271725797

In [107]:
df = DataFrame(paint_area=String[], ȧ_mean=Float64[])

push!(df, ("x > 0", -70.71999998573446))
push!(df, ("x < 0", -76.09300464248672))
push!(df, ("y > 0", -73.9617718440074))
push!(df, ("y < 0", -72.8647241824117))
push!(df, ("z > 0", -65.93337926919034))
push!(df, ("z < 0", -80.89037271725797))

,paint_area,ȧ_mean
,String,Float64
1,x > 0,-70.72
2,x < 0,-76.093
3,y > 0,-73.9618
4,y < 0,-72.8647
5,z > 0,-65.9334
6,z < 0,-80.8904


## ある場所から半径R圏内を白く塗る

- デフォルト ： `A_B = 0.04`
- 半径Rの園内： `A_B = 0.90`

In [16]:
function make_spot!(shape, thermo_params, target, R_paint, A_paint=0.9)
    @unpack A_B = thermo_params

    selected = [norm(center - target.center) < R_paint for center in shape.facets.center]
    A_B[selected] .= A_paint

    @show target.center
    @show R_paint
    @show A_paint
end;

In [17]:
t_end = orbit.T + spin.P * 150

thermo_params = ThermoParams(
    A_B   = fill(0.04, shape.num_face),
    A_TH  = 0.0,
    k     = 0.1,
    ρ     = 1270.0,
    Cp    = 600.0,
    ϵ     = 1.,
    t_bgn = 0.0,
    t_end = t_end,
    Nt    = length(0 : spin.P/72 : t_end),
    z_max = 0.6,
    Nz    = 41,
    P     = spin.P,
);

In [18]:
# make_spot!(shape, thermo_params, shape.facets[1], 100)
# make_spot!(shape, thermo_params, shape.facets[200], 100)
make_spot!(shape, thermo_params, shape.facets[500], 100)
# make_spot!(shape, thermo_params, shape.facets[800], 100)
# make_spot!(shape, thermo_params, shape.facets[1000], 100)
# make_spot!(shape, thermo_params, shape.facets[1200], 100)
# make_spot!(shape, thermo_params, shape.facets[1500], 100)

thermo_params.A_B

target.center = [93.70233333333333, 484.689, 41.068999999999996]
R_paint = 100
A_paint = 0.9


1500-element Vector{Float64}:
 0.04
 0.04
 0.04
 0.04
 0.04
 0.04
 0.04
 0.04
 ⋮
 0.04
 0.04
 0.9
 0.04
 0.04
 0.04
 0.04

In [23]:
draw(shape; data=thermo_params.A_B, colormap=:grays, strokecolor=:gray40)

GLMakie.Screen(...)

In [237]:
@time df = run_TPM(shape, orbit, spin, thermo_params)

df[:, :ȧ] = [ Astroshaper.drift_rate_ȧ([row.f_x, row.f_y, row.f_z] / RYUGU[:M], row.u, orbit) for row in eachrow(df) ]
ȧ_mean = mean( df.ȧ[@. df.t > (df.t[end] - orbit.T)] ) * 3600 * 24 * 365

164.719163 seconds (8.38 M allocations: 4.516 GiB, 0.45% gc time)


-136.60537099740372

In [238]:
df = DataFrame(target_id=Int64[], R_paint=Float64[], A_paint=Float64[], ȧ_mean=Float64[])

push!(df, (0, 0, 0.04, -138.61342928209942))  # Ryugu default

push!(df, (   1, 100, 0.9, -138.89861671358886))
push!(df, ( 100, 100, 0.9, -137.97552127948143))
push!(df, ( 200, 100, 0.9, -138.411673198803))
push!(df, ( 500, 100, 0.9, -136.27816040531908))
push!(df, ( 800, 100, 0.9, -136.60537099740372))
push!(df, (1000, 100, 0.9, -136.21877803349247))
push!(df, (1200, 100, 0.9, -135.84771978282524))
push!(df, (1500, 100, 0.9, -136.54691035865676))

,target_id,R_paint,A_paint,ȧ_mean
,Int64,Float64,Float64,Float64
1,0,0.0,0.04,-138.613
2,1,100.0,0.9,-138.899
3,100,100.0,0.9,-137.976
4,200,100.0,0.9,-138.412
5,500,100.0,0.9,-136.278
6,800,100.0,0.9,-136.605
7,1000,100.0,0.9,-136.219
8,1200,100.0,0.9,-135.848
9,1500,100.0,0.9,-136.547


In [65]:
# a = RYUGU[:a] * AU
# ȧ = (138.6 - 136.3) / (3600*24*365)
# Δt = (3600*24*365) * 100.

# - 3/2 * GM☉^(0.5) * a^(-1.5) * ȧ * Δt^2

In [106]:
# draw(shape; data=[flux.sun for flux in shape.facets.flux])
# draw(shape; data=[flux.rad for flux in shape.facets.flux])

## 熱計算の収束判定

In [18]:
# using Plots

In [11]:
# thermo_params = ThermoParams(
#     A_B   = 0.04,
#     A_TH  = 0.0,
#     k     = 0.1,
#     ρ     = 1270.0,
#     Cp    = 600.0,
#     ϵ     = 1.,
#     t_bgn = 0.0,
#     t_end = orbit.T,
#     Nt    = length(0:spin.P/72:orbit.T),
#     z_max = 0.6,
#     Nz    = 41,
#     P     = spin.P,
# )


In [19]:
# @time df = run_TPM(shape, orbit, spin, thermo_params)

In [13]:
# using Statistics

# df[:, :E_cons] = df.E_out ./ df.E_in
# df[:, :Ē_cons] = [ mean(df.E_cons[@. row.t - spin.P ≤ df.t ≤ row.t]) for row in eachrow(df) ];

In [21]:
# plot(framestyle=:box)
# plot!(legend=false)

# xs = df.t / spin.P

# plot!(xlims=(0, 500))

# plot!(xs, df.E_in, c=:orange)
# plot!(xs, df.E_out, c=:darkblue)

# plot!(xlabel="Rotation cycle")
# plot!(ylabel="Enegy input/output [W]")

In [23]:
# xs = df.t / spin.P

# p0 = plot(framestyle=:box)
# plot!(legend=:bottomright)
# plot!(xs, df.E_in,  c=:orange,   label="E_in")
# plot!(xs, df.E_out, c=:darkblue, label="E_out")
# plot!(xlabel="Rotation cycles")

# p1 = plot(p0)
# plot!(xlims=[0,15])
# plot!(yticks=[0, 2e8, 4e8, 6e8, 8e8, 1e9])
# plot!(ylabel="Enegy input/output [W]")

# p2 = plot(p0)
# plot!(xlims=[0,150])
# plot!(yticks=false)

# p3 = plot(p0)
# plot!(xlims=[0,1500])
# plot!(yticks=false)

# plot(p1, p2, p3, layout=(1,3))
# plot!(ylims=[0,1e9])
# plot!(size=(900,300))
# plot!(left_margin=4Plots.mm)
# plot!(bottom_margin=4Plots.mm)
# # savefig("enegey_IO.pdf")

In [25]:
# plot(framestyle=:box)
# plot!(legend=false)

# xs = df.t / spin.P
# plot!(xs, df.Ē_cons, lw=5, lc=:darkorange)
# hline!([1], lc=:gray)

# plot!(xlims=(0, 200))
# plot!(ylims=(0, 1.1))
# plot!(yticks=[0,0.2,0.4,0.6,0.8,1.0])

# plot!(xlabel="Rotation cycle")
# plot!(ylabel="E_out / E_in over a rotation cycle")

# plot!(size=(400,300))
# # savefig("enegey_IO_ratio.pdf")